In [ ]:
# Boter kaas en eieren bot met reinforcement learning volgens het monte-carlo principe.
# Author: Ole Timmers - S1115222

In [1]:
# Allereerst importeren we het nodige:
import numpy as np
import random

In [2]:
# Testbord - dit is hoe een bord zal worden aangemaakt.
test_bord = np.matrix([[' ',' ','o'],[' ',' ','o'],['x','x',' ']])

In [3]:
# Bereken de unieke code waarin een state kan worden vastgelegd.
# Dit gaan we gebruiken als zoekcode om de gemiddelde win-ratio vast te leggen
# in een dictionary.
# returns int

def berekenstate(bordToCalc):
    k = 0
    h = 0
    for i in range(len(bordToCalc)):
      for j in range(len(bordToCalc)):
        if bordToCalc[i,j]   == ' ':
          v = 0
        elif bordToCalc[i,j] == 'x':
          v = 1
        elif bordToCalc[i,j] == 'o':
          v = 2
        h += (3**k) * v
        k += 1
    return h

In [4]:
# Hiermee kunnen we het bord printen in de console.
def draw_board(bordToDraw):
    for i in range(len(bordToDraw)):
      print("-------------")
      for j in range(len(bordToDraw)):
        print("  ", end="")
        if bordToDraw[i,j] == 'x':
          print("x ", end="")
        elif bordToDraw[i,j] == 'o':
          print("o ", end="")
        else:
          print("  ", end="")
      print("")
    print("-------------")

In [5]:
#Testje
draw_board(test_bord)

-------------
          o 
-------------
          o 
-------------
  x   x     
-------------


In [6]:
#Test berekenstate
state = berekenstate(test_bord)
print(state)

3420


In [7]:
# Krijg alle mogelijke bewegingen te zien
# Note: Je ziet dat ik allerlei verschillende variabel-namen gebruik voor het bord.
#       Dat is in principe niet nodig natuurlijk, toch deed ik het omdat ik verschillende variabelen niet uit elkaar kon halen. XD
#       Dus in principe kan je overal 'bord' zetten.
def getMogelijkeBewegingen(bordjee):
    moves = []
    for i in range(len(bordjee)):
      for j in range(len(bordjee)):
        if bordjee[i,j] == " ":
                moves.append((i, j))
    return moves

In [8]:
#Test getMogelijkeBewegingen
print(getMogelijkeBewegingen(test_bord))

[(0, 0), (0, 1), (1, 0), (1, 1), (2, 2)]


In [9]:
# getWinner(0), functie die checkt of er een winner is. 
# Bij 1 of 2 is er een winnaar.
# Bij 0 gelijkspel
# Bij -1 zijn er nog bewegingen te maken.
def getWinner(bord):
    candidate = ' '
    won = 0
    
    # Eerst bekijken we rij voor rij
    for i in range(len(bord)):
        candidate = ' '
        for j in range(len(bord)):
            
            # Wanneer er een gat is in een rij, is er sws geen winnaar.
            if bord[i,j] == ' ':
                break
            
            # Bekijk wat er in het eerste vakje staat.
            # Want wanneer dat vakje is gevuld, moeten de rest v/d vakjes
            # hetzelfde symbool hebben.
            if candidate == ' ':
                candidate = bord[i,j]
            
            # Break op wanneer symbolen niet overeen komen.
            if candidate != bord[i,j]:
                break
            elif j == len(bord) - 1:
                # Wanneer we alle drie de vakjes hebben gehad
                # en er is niet gebreaked. Dan hebben we een winnaar.
                if candidate == 'x':
                    won = 1
                else:
                    won = 2
    
    if won > 0:
        return won
    
    # Nu doen we hetzelfde voor de kolommen.
    for j in range(len(bord)):
        candidate = ' '
        for i in range(len(bord)):
            
            # Geen gaten in de kolom - check
            if bord[i,j] == ' ':
                break
            
            # Eerste symbool opvangen
            if candidate == ' ':
                candidate = bord[i,j]
            
            # Controleren of ze allemaal gelijk zijn.
            if candidate != bord[i,j]:
                break
            elif i == len(bord) - 1:
                # We hebben een winnaar.
                if candidate == 'x':
                    won = 1
                else:
                    won = 2
    
    if won > 0:
        return won
    
    # Nu gaan we de diagonalen controleren.
    # (0,0), (1,1) & (2,2)
    candidate = ' '
    for i in range(len(bord)):
        if bord[i, i] == ' ':
            break
        if candidate == ' ':
            candidate = bord[i,i]
        if candidate != bord[i,i]:
            break
        elif i == len(bord) - 1:
            # We hebben een winnaar.
            if candidate == ' ':
                won = 1
            else:
                won = 2
    
    if won > 0:
        return won
    
    # (0,2), (1,1) (2,0)
    candidate = ""
    for i in range(len(bord)):
        if bord[i,2 - i] == ' ':
            break
        if candidate == ' ':
            candidate = bord[i , 2 - i]
        if candidate != bord[i , 2 - i]:
            break
        elif i == len(bord) - 1:
            # We hebben een winnaar.
            if candidate == 'x':
                won = 1
            else:
                won = 2
    
    if won > 0:
        return won
    
    # Nog steeds geen winnaar?
    if (len(getMogelijkeBewegingen(bord)) == 0):
        # It's a draw
        return 0
    else:
        # Nog bewegingen te maken
        return -1

In [10]:
#Test getwinner
print(getWinner(test_bord))

-1


In [11]:
# getRandomMove()
# Om een 'domme' bot te simuleren, maken wij gebruik van getRandom om 
# een willekeurige beweging uit te voeren.
def getRandomMove(bord):
    mogelijkeBewegingen = getMogelijkeBewegingen(bord)
    return random.choice(mogelijkeBewegingen)

In [12]:
# Simuleer de game - random vs random
# hiermee simuleer ik de game waar beide players random zullen gaan kiezen.
# De stappen van p1 (x) zet ik als states in de historie
# Deze zal ik verolgens returnen waarna deze in een dictionary kunnen worden gezet i.c.m. winst/verlies.

# Dit was een eerste stap in het machine learning gedeelte, omdat ik hier nog niet het reinforcement learning-gedeelte toepas.
# Later heb ik ook nog een functie waar 'ie wel ook aan reinfocement learning doet.
def simulateGameWithRandomMoves():
    historieVanStatesP1 = []
    currentbord = np.matrix([[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']])
    playerToMove = 'x'
    
    while getWinner(currentbord) == -1:
        
        # Kies random move
        move = getRandomMove(currentbord)
        
        # Doe beweging
        currentbord[move[0], move[1]] = playerToMove
        
        if playerToMove == 'x':
            # Voeg de nieuwe status van het bord (door speler 1) aan de historie
            historieVanStatesP1.append(berekenstate(currentbord))
            # switch player
            playerToMove = 'o'
        else:
            playerToMove = 'x'
            
    return (historieVanStatesP1, getWinner(currentbord))

In [21]:
# Test winratio - random vs random
# Hier test ik bovenstaande functies in een for loop en kijk ik wat de resultaten zijn. 
wins = 0
losses = 0
draws = 0

for i in range(10000):
    resultOfGame = simulateGameWithRandomMoves()
    uitslag = resultOfGame[1]
    if (uitslag == 1):
        wins += 1
    elif (uitslag == 2):
        losses += 1
    else:
        draws += 1
        
print('wins: ' + str(wins))
print('losses: ' + str(losses))
print('draws: ' + str(draws))

wins: 4431
losses: 3242
draws: 2327


In [14]:
# berekenNieuwGemiddelde(currentGemiddelde, numberToBeAdded, currentCount)
# Met deze functie pak ik het oude gemiddelde(currentgemiddelde) en het getal
# over hoeveel keer dat gemiddelde is genomen.
# Met die twee gegevens en het nieuwe resultaat, maak ik een nieuwe gemiddelde aan.

# Voor de duidelijkheid: ik bereken hier het gemiddelde van een bepaalde state die ik heb uitgevoerd nadat een potje is afgelopen.
def berekenNieuwGemiddelde(currentGemiddelde, numberToBeAdded, currentCount):
    totalNumber = currentGemiddelde * currentCount
    if numberToBeAdded == 2:
        totalNumber += 2
    elif numberToBeAdded == 1:
        totalNumber += 0
    else:
        totalNumber += 1
    newCount = currentCount + 1
    return ((totalNumber / newCount), newCount)

In [15]:
# Nadat een game is geweest, slaan wij alle states op die x heeft doorgevoerd met het resultaat van de game.
# In een dictionary, die aan deze functie wordt meegegeven, gaan wij vervolgens voor iedere state 
# dit resultaat aan de gemiddelde win-ratio toevoegen en de nieuwe ratio daarna berekenen.
# Het daadwerkelijk berekenen van de ratio (het gemiddelde) doen we in de functie hierboven.

# In deze functie itereren wij door alle states heen, en voegen/passen wij deze toe/aan in de dictionary.
def addToDictionary(dictio, historie):
    states = historie[0]
    resultVanGame = historie[1]
    resultToBeAdded = 0
    # Bij verlies niks toevoegen aan gemiddelde
    if (resultVanGame == 2):
        resultToBeAdded = 0
    elif (resultVanGame == 1): # Bij winst 2
        resultToBeAdded = 2
    else:
        resultToBeAdded = 1 # Bij gelijkspel 1

    for state in states:
        # Als de entry in de dictionary nog niet bestaat, maak 'm aan.
        if not (str(state) in dictio.keys()):
            dictio[str(state)] = berekenNieuwGemiddelde(1.0, resultToBeAdded, 1)
        else:
            dictio[str(state)] = berekenNieuwGemiddelde(dictio[str(state)][0], resultToBeAdded, dictio[str(state)][1])
        
    return dictio
    

In [16]:
# Hier komt het reinforcment learning gedeelte echt van pas.
# Bij activatie van deze functie, kijkt de code naar alle mogelijkheden.
# Voor iedere mogelijkheid, maakt hij een bord van hoe het bord er uit moet zien en converteert dit vervolgens naar een statenr.
# Dit statenr is onze zoekcode binnen de dictionary, en hiermee kunnen we de winstratio ophalen.
# De state met de hoogste winstratio wordt vervolgens als beste move gekozen.
def berekenBesteMove(bordje, dictio):
    mogelijkeBewegingen = getMogelijkeBewegingen(bordje)
    bestMove = (mogelijkeBewegingen[0], 0) # beginnen bij 0
    for mogelijkheid in mogelijkeBewegingen:
        # We nemen het bord over
        bordToCalculateIn = np.matrix([[bordje[0,0],bordje[0,1],bordje[0,2]],[bordje[1,0],bordje[1,1],bordje[1,2]],[bordje[2,0],bordje[2,1],bordje[2,2]]])
        # en voeren de move uit om naar die state te komen.
        bordToCalculateIn[mogelijkheid[0], mogelijkheid[1]] = 'x'
        # We maken het statenr
        statenr = berekenstate(bordToCalculateIn)
        # en zoeken deze op in onze dictionary
        if (not (str(statenr) in dictio.keys())):
            if (bestMove[1] < 1):
                bestMove = (mogelijkheid, 1)
        else:
            if (bestMove[1] < dictio[str(statenr)][0]):
                bestMove = (mogelijkheid, dictio[str(statenr)][0])
    return bestMove

In [17]:
# Simuleer de game - reinfocement vs random
# Zelfde als de andere simuleergame, echter zal player x hier volgens reinforcment learning de juiste keuzes maken.
# Met de paramter y kunnen wij de ratio verkennen/uitbuiten vaststellen. 
# Hoe lager hoe meer uitbuiting, hoe hoger hoe meer verkennen.

def simuleerGameWithDictionary(dictio, y, printBoard = False):
    historieVanStatesP1 = []
    currentbord = np.matrix([[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']])
    playerToMove = 'x'
    
    while getWinner(currentbord) == -1:
        bestChoice = False
        # Kies random move
        if playerToMove == 'x':
            # alleen deze regels zijn anders dan de vorige functie.
            if (random.random() > y):
                move = berekenBesteMove(currentbord, dictio)[0]
            else:
                move = getRandomMove(currentbord)
                
        else:
            move = getRandomMove(currentbord)
        
        # Doe beweging
        currentbord[move[0], move[1]] = playerToMove
        
        # Mochten we het resultaat van de move willen printen, pas de parameter aan.
        if printBoard:
            print('===========')
            draw_board(currentbord)
        
        
        if playerToMove == 'x':
            # Voeg de nieuwe status van het bord (door speler 1) aan de historie
            historieVanStatesP1.append(berekenstate(currentbord))
            # switch player
            playerToMove = 'o'
        else:
            playerToMove = 'x'
            
    return (historieVanStatesP1, getWinner(currentbord))

In [18]:
dictionaryToTestWith = {}

In [19]:
# HET TESTEN van de reinforcment learning
# Dit was nogal een gekloot tussen verkennen/uitbuiten - wat is nou beter?? - maar uiteindelijk heb ik voor het volgende gekozen:
# Eerst voeg ik aan de dictionary 150 000 games toe waarbij voor 90% aan verkennen wordt gedaan.
# Daarna doe ik het andersom, 90% uitbuiten en 10% verkennen.
# Als laatste doe ik 10 000 x een game met 100% uitbuiten om de resultaten te zien.

for i in range(150000):
    historieVanGame = simuleerGameWithDictionary(dictionaryToTestWith, 0.9)
    uitslag = historieVanGame[1]
    if (uitslag == 1):
        wins += 1
    elif (uitslag == 2):
        losses += 1
    else:
        draws += 1
    dictionaryToTestWith = addToDictionary(dictionaryToTestWith, historieVanGame)

    
for i in range(150000):
    historieVanGame = simuleerGameWithDictionary(dictionaryToTestWith, 0.1)
    uitslag = historieVanGame[1]
    if (uitslag == 1):
        wins += 1
    elif (uitslag == 2):
        losses += 1
    else:
        draws += 1
    dictionaryToTestWith = addToDictionary(dictionaryToTestWith, historieVanGame)

wins = 0
losses = 0
draws = 0

# De final test....

for i in range(10000):
    historieVanGame = simuleerGameWithDictionary(dictionaryToTestWith, 0, False)
    uitslag = historieVanGame[1]
    if (uitslag == 1):
        wins += 1
    elif (uitslag == 2):
        losses += 1
    else:
        draws += 1
    dictionaryToTestWith = addToDictionary(dictionaryToTestWith, historieVanGame)

print('wins: ' + str(wins))
print('losses: ' + str(losses))
print('draws: ' + str(draws))

wins: 9455
losses: 545
draws: 0


In [20]:
# TODO:
# Zelf tegen de machine nog spelen.